In [1]:
import torch
import clip
from PIL import Image
from transformers import RobertaTokenizer,RobertaModel
import LoadData
import torch.nn as nn

d:\anaconda\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import  os
WORKING_PATH="C:/Users/tang xiaoqing/Desktop/深度学习/注意力/pytorch-multimodal_sarcasm_detection-main/"
def load_data():
    data_set=dict()
    for dataset in ["train"]:
        file=open(os.path.join(WORKING_PATH,"text_data/",dataset+".txt"),"rb")
        for line in file:
            content=eval(line)
            image=content[0]
            sentence=content[1]
            group=content[2]
            if os.path.isfile(os.path.join(WORKING_PATH,"image_data/",image+".jpg")):
                data_set[int(image)]={"text":sentence,"group":group}
    for dataset in ["test","valid"]:
        file=open(os.path.join(WORKING_PATH,"text_data/",dataset+".txt"),"rb")
        for line in file:
            content=eval(line)
            image=content[0]
            sentence=content[1]
            group=content[3] #2
            if os.path.isfile(os.path.join(WORKING_PATH,"image_data/",image+".jpg")):
                data_set[int(image)]={"text":sentence,"group":group}
    data=open('multilabel_database/img_to_five_words.txt')
    for i in data:
        i=eval(i)
        image=i[0]
        token=i[1:]
        if os.path.isfile(os.path.join(WORKING_PATH,"image_data/",image+".jpg")):
                data_set[int(image)]['token']=token
    return data_set

In [3]:
data_set=load_data()


In [4]:
import numpy as np
np.random.seed(0)
train_index=np.random.choice(range(len(data_set)),size=20000,replace=False)
train_set=[]
test_set=[]
for i in train_index:
    train_set.append(data_set[list(data_set.keys())[i]])
for i in range(len(data_set)):
    if i not in train_index:
        test_set.append(data_set[list(data_set.keys())[i]])



In [5]:
train_set=train_set[:5000]
test_set=test_set[:200]


In [6]:
img_train=[list(data_set.keys())[i]for i in train_index]

In [7]:
img_test=[list(data_set.keys())[i] for i in range(len(data_set)) if i not in train_index]

In [8]:
text_train=[]
token_train=[]
labels_train=[]
for i in train_set:
    text_train.append(i['text'])
    token_train.append(i['token'])
    labels_train.append(i['group'])

In [9]:
text_test=[]
token_test=[]
labels_test=[]
for i in test_set:
    text_test.append(i['text'])
    token_test.append(i['token'])
    labels_test.append(i['group'])

In [10]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Softmax
def INF(B,H,W):
    '''
    生成(B*W,H,H)大小的对角线为inf的三维矩阵
    Parameters
    ----------
    B: batch
    H: height
    W: width
    '''
    return -torch.diag(torch.tensor(float("inf")).repeat(H),0).unsqueeze(0).repeat(B*W,1,1).to(device)


class CC_module(nn.Module):
    
    def __init__(self,in_dim, device):
        '''
        Parameters
        ----------
        in_dim : int
            channels of input
        '''
        super(CC_module, self).__init__()
        self.query_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim//2, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim//2, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim, kernel_size=1)
        self.softmax = Softmax(dim=3)
        self.INF = INF
        self.gamma = nn.Parameter(torch.zeros(1)).to(device)
        self.device = device
          
    def forward(self, x):

        m_batchsize, _, height, width = x.size()
        
        proj_query = self.query_conv(x) #size = (b,c2,h,w), c1 = in_dim, c2 = c1 // split
        
        #size = (b*w, h, c2)
        proj_query_H = proj_query.permute(0,3,1,2).contiguous().view(m_batchsize*width,-1,height).permute(0, 2, 1) 
        
        #size = (b*h, w, c2)
        proj_query_W = proj_query.permute(0,2,1,3).contiguous().view(m_batchsize*height,-1,width).permute(0, 2, 1)
        
        proj_key = self.key_conv(x) #size = (b,c2,h,w)
        
        proj_key_H = proj_key.permute(0,3,1,2).contiguous().view(m_batchsize*width,-1,height) 
        proj_key_W = proj_key.permute(0,2,1,3).contiguous().view(m_batchsize*height,-1,width) #size = (b*w,c2,h)
        
        proj_value = self.value_conv(x) #size = (b,c1,h,w)
        proj_value_H = proj_value.permute(0,3,1,2).contiguous().view(m_batchsize*width,-1,height) #size = (b*w,c1,h)
        proj_value_W = proj_value.permute(0,2,1,3).contiguous().view(m_batchsize*height,-1,width) #size = (b*h,c1,w)
        
        #size = (b*w, h,h) ,其中[:,i,j]表示Q所有W的第Hi行的所有通道值与K上所有W的第Hj列的所有通道值的向量乘积 
        energy_H = torch.bmm(proj_query_H, proj_key_H).to(device)
        
        #size = (b,h,w,h) #这里为什么加 INF并没有理解
        energy_H = (energy_H + self.INF(m_batchsize, height, width)).view(m_batchsize,width,height,height).permute(0,2,1,3)
        
        #size = (b*h,w,w),其中[:,i,j]表示Q所有H的第Wi行的所有通道值与K上所有H的第Wj列的所有通道值的向量乘积
        energy_W = torch.bmm(proj_query_W, proj_key_W)
        energy_W = energy_W.view(m_batchsize,height,width,width) #size = (b,h,w,w)
        
        concate = self.softmax(torch.cat([energy_H, energy_W], 3)) #size = (b,h,w,h+w) #softmax归一化
        #concate = concate * (concate>torch.mean(concate,dim=3,keepdim=True)).float()
        att_H = concate[:,:,:,0:height].permute(0,2,1,3).contiguous().view(m_batchsize*width,height,height) #size = (b*w,h,h)
        #print(concate)
        #print(att_H) 
        att_W = concate[:,:,:,height:height+width].contiguous().view(m_batchsize*height,width,width) #size = (b*h,w,w)
        
        #size = (b*w,c1,h) #[:,i,j]表示V所有W的第Ci行通道上的所有H 与att_H的所有W的第Hj列的h权重的乘积  
        out_H = torch.bmm(proj_value_H, att_H.permute(0, 2, 1))
        out_H = out_H.view(m_batchsize,width,-1,height).permute(0,2,3,1)  #size = (b,c1,h,w)
        
        #size = (b*h,c1,w) #[:,i,j]表示V所有H的第Ci行通道上的所有W 与att_W的所有H的第Wj列的W权重的乘积  
        out_W = torch.bmm(proj_value_W, att_W.permute(0, 2, 1))
        out_W = out_W.view(m_batchsize,height,-1,width).permute(0,2,1,3) #size = (b,c1,h,w)
        #print(out_H.size(),out_W.size())
        
        return self.gamma*(out_H + out_W) + x 


In [11]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


class CBAM(nn.Module):
    def __init__(self, in_channel):
        super(CBAM, self).__init__()
        self.Cam = ChannelAttentionModul(in_channel=in_channel)  # 通道注意力模块
        self.Sam = SpatialAttentionModul(in_channel=in_channel)  # 空间注意力模块

    def forward(self, x):
        x = self.Cam(x)
        x = self.Sam(x)
        return x


class ChannelAttentionModul(nn.Module):  # 通道注意力模块
    def __init__(self, in_channel, r=0.5):  # channel为输入的维度, r为全连接层缩放比例->控制中间层个数
        super(ChannelAttentionModul, self).__init__()
        # 全局最大池化
        self.MaxPool = nn.AdaptiveMaxPool2d(1)

        self.fc_MaxPool = nn.Sequential(
            nn.Linear(in_channel, int(in_channel * r)),  # int(channel * r)取整数, 中间层神经元数至少为1, 如有必要可设为向上取整
            nn.ReLU(),
            nn.Linear(int(in_channel * r), in_channel),
            nn.Sigmoid(),
        )

        # 全局均值池化
        self.AvgPool = nn.AdaptiveAvgPool2d(1)

        self.fc_AvgPool = nn.Sequential(
            nn.Linear(in_channel, int(in_channel * r)),  # int(channel * r)取整数, 中间层神经元数至少为1, 如有必要可设为向上取整
            nn.ReLU(),
            nn.Linear(int(in_channel * r), in_channel),
            nn.Sigmoid(),
        )

        # 激活函数
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # 1.最大池化分支
        max_branch = self.MaxPool(x)
        # 送入MLP全连接神经网络, 得到权重
        max_in = max_branch.view(max_branch.size(0), -1)
        max_weight = self.fc_MaxPool(max_in)

        # 2.全局池化分支
        avg_branch = self.AvgPool(x)
        # 送入MLP全连接神经网络, 得到权重
        avg_in = avg_branch.view(avg_branch.size(0), -1)
        avg_weight = self.fc_AvgPool(avg_in)

        # MaxPool + AvgPool 激活后得到权重weight
        weight = max_weight + avg_weight
        weight = self.sigmoid(weight)

        # 将维度为b, c的weight, reshape成b, c, 1, 1 与 输入x 相乘
        h, w = weight.shape
        # 通道注意力Mc
        Mc = torch.reshape(weight, (h, w, 1, 1))

        # 乘积获得结果
        x = Mc * x

        return x


class SpatialAttentionModul(nn.Module):  # 空间注意力模块
    def __init__(self, in_channel):
        super(SpatialAttentionModul, self).__init__()
        self.conv = nn.Conv2d(2, 1, 7, padding=3)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x维度为 [N, C, H, W] 沿着维度C进行操作, 所以dim=1, 结果为[N, H, W]
        MaxPool = torch.max(x, dim=1).values  # torch.max 返回的是索引和value， 要用.values去访问值才行！
        AvgPool = torch.mean(x, dim=1)

        # 增加维度, 变成 [N, 1, H, W]
        MaxPool = torch.unsqueeze(MaxPool, dim=1)
        AvgPool = torch.unsqueeze(AvgPool, dim=1)

        # 维度拼接 [N, 2, H, W]
        x_cat = torch.cat((MaxPool, AvgPool), dim=1)  # 获得特征图

        # 卷积操作得到空间注意力结果
        x_out = self.conv(x_cat)
        Ms = self.sigmoid(x_out)

        # 与原图通道进行乘积
        x = Ms * x

        return x


In [12]:

import torch as pt
import torch.nn.functional as F
class MLP(pt.nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.fc1 = pt.nn.Linear(6*512,512)
        self.fc2 = pt.nn.Linear(512,128)
        self.fc3 = pt.nn.Linear(128,2)
        self.softmax=nn.Softmax(dim=1)
    def forward(self,din):
        din = din.view(-1,6*512)
        dout = pt.nn.functional.relu(self.fc1(din))
        dout = pt.nn.functional.relu(self.fc2(dout))
        return self.softmax(self.fc3(dout))



In [13]:
model1, preprocess = clip.load('ViT-B/16',device="cuda") 
model1

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [14]:

vocab_file = 'model/vocab.json'
merges_file = 'model/merges.txt'
tokenizer = RobertaTokenizer(vocab_file, merges_file)
roberta = RobertaModel.from_pretrained('model/roberta-base/')
roberta.pooler.dense=nn.Linear(in_features=768,out_features=512,bias=True)


Some weights of the model checkpoint at model/roberta-base/ were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
import torch as pt
device = torch.device('cuda:0' if torch.cuda.device_count()==1 else 'cpu')
class Model(pt.nn.Module):
    def __init__(self,indim,device,outdim):
        super(Model,self).__init__()
        self.model1=roberta.to(device)
        self.model2=model1.to(device)
        self.attention1=CC_module(in_dim=indim,device=device).to(device)
        self.attention2=CBAM(in_channel=indim).to(device)
        self.attention3=CBAM(in_channel=outdim).to(device)
        self.model3=MLP().to(device)
    def forward(self,text,token,img,num,count):
        encoded_input=tokenizer(text, return_tensors='pt',padding=True).to(device)
        text_list=self.model1(**encoded_input)[1]
        token_list=[]
        for i in token:
            text1 = clip.tokenize(i).to(device)
            text_features = self.model2.encode_text(text1)
            token_list.append(text_features)
        token_list=torch.tensor([item.cpu().detach().numpy()for item in token_list]).to(device)
        token_list=torch.max(token_list,dim=1)
        token_list=token_list.values
        image_list=[]
        for i in range(count*num,(count+1)*num):
            b=os.path.join(WORKING_PATH,"image_data/",str(img[i])+".jpg")
            if os.path.isfile(b):
                a=Image.open(b)
                image = preprocess(a).unsqueeze(0).to(device)
                image_features = self.model2.encode_image(image)
                image_list.append(image_features)
                a.close()
        image_list=torch.tensor([item.cpu().detach().numpy()for item in image_list]).to(device)
        img_list=image_list.reshape(len(image_list),512)
        x1=torch.stack([token_list,img_list],1).to(device)
        x1=x1.float()
        y1=x1.reshape(num,2,1,512)
        out1 = self.attention1(y1)
        x2=torch.stack([token_list,text_list],1)
        y2=x2.reshape(num,2,1,512)
        out2 = self.attention1(y2)
        x3=torch.stack([img_list,text_list],1)
        y3=x3.reshape(num,2,1,512)
        out3= self.attention1(y3)
        z1=torch.stack([out1[:,0],out2[:,0]],1)
        z1=z1.reshape(num,2,1,512)
        outputs1 = self.attention2(z1)
        z2=torch.stack([out1[:,1],out3[:,0]],1)
        z2=z2.expand(num,2,1,512)
        outputs2 = self.attention2(z2)
        z3=torch.stack([out2[:,1],out3[:,1]],1)
        z3=z3.expand(num,2,1,512)
        outputs2 = self.attention2(z3)
        inputs=torch.concat((outputs1,outputs2,outputs2),dim=1)
        inputs = self.attention3(inputs)
        outputs=self.model3(inputs)
        return  outputs



In [16]:
Model(indim=2,device=device,outdim=6)

Model(
  (model1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=

In [18]:
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score
loss_fn=torch.nn.CrossEntropyLoss()
model_all=Model(indim=2,device=device,outdim=6)
optimizer = torch.optim.SGD(model_all.parameters(), lr=0.01,momentum=0.9)
loss_train=[]
loss_test=[]
pres=[]
recalls=[]
avcs=[]
f1s=[]
avcs_train=[]
split=16
for j in range(3):
        for x in range(len(train_set)//split):
                labels1=labels_train[split*x:split*(x+1)]
                labels1=torch.tensor(labels1).to(device)
                outputs =model_all(text_train[x*split:(x+1)*split],token_train[x*split:(x+1)*split],img_train,split,x)
                pred = torch.argmax(outputs,1)
                loss = loss_fn(outputs,labels1)
                loss_train.append(loss)
                optimizer.zero_grad()
                loss.backward()                
                optimizer.step()
                pred=pred.cpu()
                labels1=labels1.cpu()
                avcs_train.append(accuracy_score(labels1,pred))
                print(accuracy_score(labels1,pred))
torch.save(model_all,'MODEL.pt')
model_all= torch.load('MODEL.pt')
model_all.eval()
with torch.no_grad():
        for h in range(len(test_set)//split):
                labels2=labels_test[h*split:(h+1)*split]
                labels2=torch.tensor(labels2).to(device)
                outputs =model_all(text_test[h*split:(h+1)*split],token_test[h*split:(h+1)*split],img_test,split,h)
                pred = torch.argmax(outputs,1)
                loss = loss_fn(outputs,labels2)
                loss_test.append(loss)
                labels2=labels2.cpu()
                pred=pred.cpu()
                f1s.append(f1_score(labels2,pred))
                pres.append(precision_score(labels2,pred))
                recalls.append(recall_score(labels2,pred))
                avcs.append(accuracy_score(labels2,pred))
print(np.mean(pres),np.mean(recalls),np.mean(recalls),np.mean(avcs))

0.5625
0.5
0.4375
0.625
0.4375
0.5625
0.625
0.625
0.5
0.4375
0.5625
0.5625
0.4375
0.5
0.625
0.4375
0.8125
0.5625
0.625
0.5
0.75
0.5625
0.375
0.5
0.6875
0.5625
0.8125
0.5
0.5625
0.75
0.625
0.75
0.5
0.6875
0.625
0.6875
0.5625
0.6875
0.5
0.4375
0.4375
0.6875
0.5625
0.6875
0.8125
0.625
0.625
0.5625
0.75
0.8125
0.375
0.5
0.625
0.5
0.5625
0.5625
0.5625
0.6875
0.75
0.4375
0.5625
0.4375
0.75
0.5625
0.75
0.25
0.75
0.625
0.5
0.5625
0.5
0.5
0.375
0.375
0.4375
0.75
0.5
0.4375
0.375
0.5625
0.5625
0.5625
0.6875
0.6875
0.75
0.625
0.625
0.3125
0.3125
0.75
0.5
0.5625
0.5625
0.5625
0.5
0.4375
0.8125
0.375
0.6875
0.6875
0.5625
0.5625
0.6875
0.5625
0.5625
0.4375
0.5
0.5625
0.375
0.5625
0.4375
0.5
0.5625
0.375
0.625
0.5
0.8125
0.75
0.5625
0.375
0.5
0.625
0.375
0.6875
0.5
0.625
0.6875
0.625
0.625
0.625
0.375
0.25
0.75
0.625
0.5
0.5
0.625
0.25
0.5625
0.625
0.5625
0.5625
0.3125
0.6875
0.5625
0.5625
0.25
0.5625
0.4375
0.4375
0.75
0.625
0.5625
0.625
0.6875
0.5625
0.8125
0.5625
0.5625
0.75
0.6875
0.5
0.5625
0.56

RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 602112 bytes.